<a href="https://colab.research.google.com/github/ShesterG/Stance-Detection-Ghana-2020-Elections/blob/main/Notebooks/Best_Model_LR_3Classes_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression Model for Twitter Stance Analysis (3 Classes)

# Imports and Constants

In [1]:
!git clone https://github.com/ShesterG/Stance-Detection-Ghana-2020-Elections.git

%cd Stance-Detection-Ghana-2020-Elections/

fatal: destination path 'Stance-Detection-Ghana-2020-Elections' already exists and is not an empty directory.
/content/Stance-Detection-Ghana-2020-Elections


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, plot_confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.preprocessing import LabelBinarizer
#import evaluation_functions
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer 
from imblearn.over_sampling import SMOTE
import ast



from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import StratifiedKFold


from sklearn.metrics import make_scorer

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
DATA_FILE_PATH = '/content/Stance-Detection-Ghana-2020-Elections/Dataset/'
CLEAN_DATA_FILE_NAME = '01EDA_All_Model_Tweets_Ghana_2020_Elections_L.csv'

ALL_TOKENIZED_DATA_FILE_NAME = '02TPP_All_Tweets_Ghana_2020_Elections_L.csv'

TOKENIZED_DATA_FILE_NAME = '02TPP_All_Model_Tweets_Ghana_2020_Elections_L.csv'
pd.set_option('display.max_colwidth', None)
RANDOM_STATE = 42

# Load Data

In [5]:
df = pd.read_csv(DATA_FILE_PATH + TOKENIZED_DATA_FILE_NAME)
df_ALL = pd.read_csv(DATA_FILE_PATH + ALL_TOKENIZED_DATA_FILE_NAME)

In [6]:
df.head()

,ID,Tweet,Candidate,Stance,Preproc_Tweet,Tokens
0,2,"@NAkufoAddo H. E Nana please follow up on this FAKE NEWS, LIES and sue BBC for peddling false information. Hold alÃ¢â¬Â¦ https://t.co/SdwnMsxeUV",Nana,0.0,nana h e nana please follow up on this fake news lies and sue bbc for peddling false information hold al,"['nana', 'h', 'e', 'nana', 'pleas', 'follow', 'up', 'on', 'thi', 'fake', 'news', 'lie', 'and', 'sue', 'bbc', 'for', 'peddl', 'fals', 'inform', 'hold', 'al']"
1,6,"@ChipsJhay @TruthFreema @AOuattara_PRCI @NAkufoAddo No, you haven't. He's using his freedom of speech as it is. He's covered.",Nana,0.0,nana no you haven't he's using his freedom of speech as it is he's covered,"['nana', 'no', 'you', ""haven't"", ""he'"", 'use', 'hi', 'freedom', 'of', 'speech', 'as', 'it', 'is', ""he'"", 'cover']"
2,7,@IssahGerard @HEgyiri @NAkufoAddo Please who is this and how is he your in law??,Nana,0.0,nana please who is this and how is he your in law,"['nana', 'pleas', 'who', 'is', 'thi', 'and', 'how', 'is', 'he', 'your', 'in', 'law']"
3,17,@Kwame34580189 @bukolasaraki @NAkufoAddo Whatever.... Thought you had something there....,Nana,0.0,nana whatever thought you had something there,"['nana', 'whatev', 'thought', 'you', 'had', 'someth', 'there']"
4,18,@NAkufoAddo Remember that the good people of Gh have spoken therefore Soo be it,Nana,0.0,nana remember that the good people of gh have spoken therefore soo be it,"['nana', 'rememb', 'that', 'the', 'good', 'peopl', 'of', 'gh', 'have', 'spoken', 'therefor', 'soo', 'be', 'it']"


In [7]:
#df = df[df.Stance != 0]

In [8]:
df.Tokens = df.Tokens.map(ast.literal_eval)
df_ALL.Tokens = df_ALL.Tokens.map(ast.literal_eval)

In [9]:
data = df[['Tokens', 'Stance']]
data_ALL = df_ALL[['Tokens', 'Stance']]

In [10]:
df.value_counts(["Candidate", "Stance"])

Candidate  Stance
Nana        0.0      803
John       -1.0      515
            1.0      515
Nana       -1.0      515
            1.0      515
John        0.0      227
dtype: int64

In [11]:
data.Stance.value_counts()

 0.0    1030
 1.0    1030
-1.0    1030
Name: Stance, dtype: int64

In [12]:
data.head()

,Tokens,Stance
0,"[nana, h, e, nana, pleas, follow, up, on, thi, fake, news, lie, and, sue, bbc, for, peddl, fals, inform, hold, al]",0.0
1,"[nana, no, you, haven't, he', use, hi, freedom, of, speech, as, it, is, he', cover]",0.0
2,"[nana, pleas, who, is, thi, and, how, is, he, your, in, law]",0.0
3,"[nana, whatev, thought, you, had, someth, there]",0.0
4,"[nana, rememb, that, the, good, peopl, of, gh, have, spoken, therefor, soo, be, it]",0.0


# Split Data

In [13]:
X = data['Tokens']
y = data['Stance']

X_ALL = data_ALL['Tokens']

# Encode Labels

In [14]:
le = LabelEncoder()

In [15]:
y_enc = le.fit_transform(y)

# Evaluate Adjustments to Minimum Document Frequency

In [16]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer:
    def __init__(self):
         self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in doc]

In [17]:
class stop_word_remover:
    def __init__(self):
         self.stop_words = frozenset(['the', 'to', 'at', 'rt', 'for', 'a', 
                                      'in', 'is', 'of', 'and', 'on', 'i', 
                                      'you', 'an', 'with', 'it', 'this'])
    def __call__(self, doc):
        return [t for t in doc if t not in self.stop_words]

In [18]:
def passthrough(doc):
    """passthrough function for use in the pipeline because the text is already tokenized"""
    return doc

In [19]:
pipeline = Pipeline([
                        ('bow',CountVectorizer(min_df=1,
                                               ngram_range=(1,2),
                                               max_features=10000,
                                               analyzer='word',
                                               preprocessor=stop_word_remover(),
                                               tokenizer=LemmaTokenizer(), 
                                               lowercase=False)),  
                        ('tfidf', TfidfTransformer()),
                        ('smote', SMOTE(random_state=RANDOM_STATE, n_jobs=-1)),
                        ('lr_classifier', LogisticRegression(random_state=RANDOM_STATE))])

pipeline.fit(X, y_enc)
df_ALL["Predictions"] = pipeline.predict(X_ALL)





In [20]:
df_ALL.groupby(["Candidate", "Predictions"]).size().reset_index(name="#Tweets") 

# 0, 1, 2   --->   -1, 0, 1 

,Candidate,Predictions,#Tweets
0,John,0,15361
1,John,1,4984
2,John,2,6183
3,Nana,0,15504
4,Nana,1,43912
5,Nana,2,13412
